In [1]:
import xarray as xr
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
url = 'http://psl.noaa.gov/thredds/dodsC/Projects/CEFI/regional_mom6/northwest_atlantic/forecast/tob_forecast_i199303.nc'
forecast_test = xr.open_dataset(url)

OSError: [Errno -68] NetCDF: I/O failure: 'http://psl.noaa.gov/thredds/dodsC/Projects/CEFI/regional_mom6/northwest_atlantic/forecast/tob_forecast_i199303.nc'